In [1]:
%%javascript
if(IPython.tab_as_tab_everywhere)IPython.tab_as_tab_everywhere()

<IPython.core.display.Javascript object>

In [2]:
import matplotlib.pyplot as plt
import os, sys, json
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy import stats
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import InputLayer, Input
from tensorflow.python.keras.layers import Reshape, MaxPooling2D,Dropout
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.optimizers import Adam
from sklearn.metrics import roc_auc_score, f1_score


pd.set_option('display.max_rows', 25)
pd.set_option('display.max_columns', 200)

/usr/local/lib/python3.5/dist-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/usr/local/lib/python3.5/dist-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [3]:
train = pd.read_csv('data/train.csv', encoding='ISO-8859-1').sample(frac=1)
test = pd.read_csv('data/train.csv', encoding='ISO-8859-1')

In [4]:
train[:10]

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,r4h3,r4m1,r4m2,r4m3,r4t1,r4t2,r4t3,tamhog,tamviv,escolari,rez_esc,hhsize,paredblolad,paredzocalo,paredpreb,pareddes,paredmad,paredzinc,paredfibras,paredother,pisomoscer,pisocemento,pisoother,pisonatur,pisonotiene,pisomadera,techozinc,techoentrepiso,techocane,techootro,cielorazo,abastaguadentro,abastaguafuera,abastaguano,public,planpri,noelec,coopele,sanitario1,sanitario2,sanitario3,sanitario5,sanitario6,energcocinar1,energcocinar2,energcocinar3,energcocinar4,elimbasu1,elimbasu2,elimbasu3,elimbasu4,elimbasu5,elimbasu6,epared1,epared2,epared3,etecho1,etecho2,etecho3,eviv1,eviv2,eviv3,dis,male,female,estadocivil1,estadocivil2,estadocivil3,estadocivil4,estadocivil5,estadocivil6,estadocivil7,parentesco1,parentesco2,parentesco3,parentesco4,parentesco5,parentesco6,parentesco7,parentesco8,parentesco9,parentesco10,parentesco11,parentesco12,idhogar,hogar_nin,hogar_adul,hogar_mayor,hogar_total,dependency,edjefe,edjefa,meaneduc,instlevel1,instlevel2,instlevel3,instlevel4,instlevel5,instlevel6,instlevel7,instlevel8,instlevel9,bedrooms,overcrowding,tipovivi1,tipovivi2,tipovivi3,tipovivi4,tipovivi5,computer,television,mobilephone,qmobilephone,lugar1,lugar2,lugar3,lugar4,lugar5,lugar6,area1,area2,age,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
7608,ID_9d9baafaf,40000.0,0,6,0,1,1,0,NaN,1,0,1,0,2,2,1,2,3,3,3,12,NaN,3,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,d73a97383,1,2,0,3,.5,no,15,13.500000,0,0,0,0,0,0,0,1,0,3,1.000000,0,0,1,0,0,0,0,1,2,0,0,0,1,0,0,0,1,21,144,441,9,0,1,1.000000,0.250000,182.25000,441,4
2224,ID_30c5ef107,NaN,0,5,0,1,1,0,NaN,0,1,1,2,1,3,2,2,4,4,4,0,NaN,4,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,c0ab646bb,2,2,0,4,yes,no,15,14.500000,1,0,0,0,0,0,0,0,0,3,1.333333,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,1,0,1,16,0,4,1.777778,1.000000,210.25000,1,4
6198,ID_34ef645e4,NaN,0,5,0,1,1,0,NaN,1,3,4,0,1,1,1,4,5,5,5,6,NaN,5,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,cbe81b20d,2,3,0,5,.66666669,6,no,10.333333,0,0,1,0,0,0,0,0,0,3,1.666667,1,0,0,0,0,0,1,1,2,0,1,0,0,0,0,0,1,47,36,2209,25,36,4,2.777778,0.444444,106.77777,2209,4
1209,ID_8451eca26,245000.0,0,4,0,1,1,1,2.0,2,1,3,0,1,1,2,2,4,4,4,15,NaN,4,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,ed256aa01,2,2,0,4,yes,no,15,15.000000,0,0,0,0,0,0,0,1,0,3,1.333333,0,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,29,225,841,16,0,4,1.777778,1.000000,225.00000,841,4
5801,ID_c585dea81,NaN,0,4,0,1,1,0,NaN,1,2,3,0,2,2,1,4,5,5,5,9,NaN,5,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,eef1a4803,2,3,0,5,.66666669,9,no,12.666667,0,0,0,1,0,0,0,0,0,3,1.666667,1,0,0,0,0,0,1,1,4,0,1,0,0,0,0,1,0,19,81,361,25,81,4,2.777778,0.444444,160.44446,361,4
2306,ID_645c04e40,NaN,0,4,0,1,1,0,NaN,0,1,1,0,1,1,0,2,2,2,2,14,NaN,2,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,379646ab0,0,2,1,2,yes,14,no,14.000000,0,0,0,0,0,0,0,1,0,2,1.000000,1,0,0,0,0,0,0,1,2,1,0,0,0,0,0,1,0,68,196,4624,4,196,0,1.000000,1.000000,196.00000,4624,4
7923,ID_29a7f7ea6,NaN,0,3,0,1,1,0,NaN,1,1,2,0,1,1,1,2,3,3,3,3,NaN,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,f8d3c34f4,1,2,0,3,.5,3,no,6.000000,0,1,0,0,0,0,0,0,0,1,3.000000,0,0,0,0,1,0,0,1,2,0,0,0,0,1,0,1,0,24,9,576,9,9,1,9.000000,0.250000,36.00000,576,1
104,ID_4cf6a7097,NaN,0,4,0,1,1,1,1.0,1,1,2,0,1,1,1,2,3,3,3

In [5]:
drop = ['Id','idhogar','elimbasu5','Target']
#probably useless features
#drop +=['planpri','paredother','coopele','estadocivil6','estadocivil7','dependency-yes','hogar_mayor','instlevel6','parentesco7','tipovivi1','tipovivi3','parentesco8','public','parentesco10','parentesco9','sanitario6','parentesco12','instlevel4','meaneduc-nan','SQBmeaned-nan','parentesco5','lugar2','parentesco11','techoentrepiso']
ids = ['Extreme Poverty','Moderate Poverty','Vulnerable Households','Non Vulnerable Households']

In [6]:
def build_structure():
	uniques = {}
	scale = {}
	for col in train:
		if col in drop: continue
	
		num = pd.to_numeric(train[col], errors='coerce')
		null = np.isnan(num)
		column = np.nan_to_num(num.copy(),0)
		d_min = float(column.min())
		d_max = float(column.max())
		dif = d_max - d_min
		
		scale[col] = {
			'a': d_min,
			'b': dif,
		}
		
		if not np.all(~null):
			data_col = train[col].fillna('nan')
			unique = data_col[null].unique()
			uniques[col] = unique
	


	
	return {
		'uniques':uniques,
		'scale': scale
	}
struct = build_structure()

In [7]:
def get_normalize(train,struct):
	df = pd.DataFrame()
	for col in train:
		if col in drop: continue
		num = pd.to_numeric(train[col], errors='coerce')
		null = np.isnan(num)
		df[col] = (np.nan_to_num(num.copy(),0) - struct['scale'][col]['a']) / struct['scale'][col]['b']
		
		if col in struct['uniques']:
			data_col = train[col].fillna('nan')
			for key in struct['uniques'][col]:
				df[col+ '-' + key] = (data_col == key).tolist()
		
		
	return df


In [8]:
df = get_normalize(train,struct)

In [9]:
one_hot = pd.get_dummies(train['Target'])

columns = len(df.columns)
train_percent = 0.9
train_num = int(len(train) * train_percent)
test_num = len(train) - train_num

train_x = df.head(train_num).as_matrix()
train_y = one_hot.head(train_num).as_matrix()

test_x = df.tail(test_num).as_matrix()
test_y = one_hot.tail(test_num).as_matrix()

In [10]:
inputs = Input(shape=(columns,))

net = inputs
w = 150
h = 3
for _ in range(h):
	net = Dense(w, activation='relu')(net)

net = Dropout(0.5)(net)

net = Dense(len(ids), activation='softmax')(net)

# Output of the Neural Network.
outputs = net
model = Model(inputs=inputs, outputs=outputs)

In [11]:
model.compile(optimizer='sgd', loss='categorical_crossentropy')

In [12]:
model.fit(x=train_x, y=train_y,validation_split=0.5,epochs=1000, batch_size=128)

Train on 4300 samples, validate on 4301 samples
Epoch 1/1000
4300/4300 [==============================] - 0s 112us/step - loss: 1.1804 - val_loss: 1.0322
Epoch 2/1000
4300/4300 [==============================] - 0s 59us/step - loss: 1.0597 - val_loss: 1.0042
Epoch 3/1000
4300/4300 [==============================] - 0s 59us/step - loss: 1.0421 - val_loss: 0.9912
Epoch 4/1000
4300/4300 [==============================] - 0s 58us/step - loss: 1.0211 - val_loss: 0.9793
Epoch 5/1000
4300/4300 [==============================] - 0s 59us/step - loss: 1.0089 - val_loss: 0.9687
Epoch 6/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.9935 - val_loss: 0.9599
Epoch 7/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.9904 - val_loss: 0.9507
Epoch 8/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.9723 - val_loss: 0.9432
Epoch 9/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.9663 - val_loss: 0.9372
Epoch 

Epoch 78/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.8313 - val_loss: 0.8464
Epoch 79/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.8224 - val_loss: 0.8451
Epoch 80/1000
4300/4300 [==============================] - 0s 58us/step - loss: 0.8237 - val_loss: 0.8436
Epoch 81/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.8242 - val_loss: 0.8450
Epoch 82/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.8237 - val_loss: 0.8431
Epoch 83/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.8221 - val_loss: 0.8421
Epoch 84/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.8191 - val_loss: 0.8424
Epoch 85/1000
4300/4300 [==============================] - 0s 58us/step - loss: 0.8249 - val_loss: 0.8458
Epoch 86/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.8163 - val_loss: 0.8412
Epoch 87/1000
4300/4300 [=====================

Epoch 155/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.7449 - val_loss: 0.8082
Epoch 156/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.7430 - val_loss: 0.8084
Epoch 157/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.7400 - val_loss: 0.8092
Epoch 158/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.7438 - val_loss: 0.8079
Epoch 159/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.7363 - val_loss: 0.8069
Epoch 160/1000
4300/4300 [==============================] - 0s 58us/step - loss: 0.7399 - val_loss: 0.8091
Epoch 161/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.7357 - val_loss: 0.8069
Epoch 162/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.7352 - val_loss: 0.8053
Epoch 163/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.7344 - val_loss: 0.8074
Epoch 164/1000
4300/4300 [===========

4300/4300 [==============================] - 0s 59us/step - loss: 0.5296 - val_loss: 1.0210
Epoch 308/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.5165 - val_loss: 0.7791
Epoch 309/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.5347 - val_loss: 0.8967
Epoch 310/1000
4300/4300 [==============================] - 0s 58us/step - loss: 0.5475 - val_loss: 0.7807
Epoch 311/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.4933 - val_loss: 0.7856
Epoch 312/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.5201 - val_loss: 1.0571
Epoch 313/1000
4300/4300 [==============================] - 0s 60us/step - loss: 0.5142 - val_loss: 0.7988
Epoch 314/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.5075 - val_loss: 0.8372
Epoch 315/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.5700 - val_loss: 0.7870
Epoch 316/1000
4300/4300 [==========================

4300/4300 [==============================] - 0s 60us/step - loss: 0.3385 - val_loss: 0.8400
Epoch 460/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.2554 - val_loss: 0.8709
Epoch 461/1000
4300/4300 [==============================] - 0s 60us/step - loss: 0.3749 - val_loss: 0.8462
Epoch 462/1000
4300/4300 [==============================] - 0s 62us/step - loss: 0.2500 - val_loss: 1.0399
Epoch 463/1000
4300/4300 [==============================] - 0s 62us/step - loss: 0.2722 - val_loss: 0.8719
Epoch 464/1000
4300/4300 [==============================] - 0s 61us/step - loss: 0.2806 - val_loss: 0.8462
Epoch 465/1000
4300/4300 [==============================] - 0s 62us/step - loss: 0.3876 - val_loss: 0.9252
Epoch 466/1000
4300/4300 [==============================] - 0s 60us/step - loss: 0.2687 - val_loss: 0.8313
Epoch 467/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.2689 - val_loss: 1.0045
Epoch 468/1000
4300/4300 [==========================

4300/4300 [==============================] - 0s 60us/step - loss: 0.1027 - val_loss: 1.0070
Epoch 612/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.1125 - val_loss: 1.0292
Epoch 613/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.0996 - val_loss: 1.0268
Epoch 614/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.2228 - val_loss: 1.0047
Epoch 615/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.1054 - val_loss: 1.0137
Epoch 616/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.0973 - val_loss: 1.0172
Epoch 617/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.0978 - val_loss: 1.0392
Epoch 618/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.1037 - val_loss: 1.1691
Epoch 619/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.0992 - val_loss: 1.0375
Epoch 620/1000
4300/4300 [==========================

4300/4300 [==============================] - 0s 60us/step - loss: 0.0395 - val_loss: 1.2225
Epoch 764/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.0370 - val_loss: 1.2339
Epoch 765/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.3821 - val_loss: 1.1506
Epoch 766/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.0487 - val_loss: 1.1634
Epoch 767/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.0402 - val_loss: 1.1827
Epoch 768/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.0401 - val_loss: 1.2066
Epoch 769/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.0375 - val_loss: 1.2033
Epoch 770/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.0361 - val_loss: 1.2227
Epoch 771/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.0371 - val_loss: 1.2209
Epoch 772/1000
4300/4300 [==========================

4300/4300 [==============================] - 0s 59us/step - loss: 0.0205 - val_loss: 1.4508
Epoch 916/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.0186 - val_loss: 1.3880
Epoch 917/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.0169 - val_loss: 1.4041
Epoch 918/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.0189 - val_loss: 1.3992
Epoch 919/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.0172 - val_loss: 1.4040
Epoch 920/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.0198 - val_loss: 1.4007
Epoch 921/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.0188 - val_loss: 1.4140
Epoch 922/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.0177 - val_loss: 1.4120
Epoch 923/1000
4300/4300 [==============================] - 0s 59us/step - loss: 0.0160 - val_loss: 1.4026
Epoch 924/1000
4300/4300 [==========================

In [13]:
predictions = model.predict(test_x)
guess = np.argmax(predictions,axis=1) + 1
correct = np.argmax(test_y,axis=1) + 1

In [14]:
f1_score(correct, guess, average='macro')

0.6001643783296586

In [15]:
total = 0
for i in range(4):
	total+=roc_auc_score(test_y[:,i],predictions[:,i])
total / 4

0.860260954827055

In [16]:
test_df = get_normalize(test,struct)
test_predictions = np.argmax(model.predict(test_df.as_matrix()),axis=1) + 1

In [17]:
output = pd.DataFrame()
output['Id'] = test['Id']
output['Target'] = test_predictions
output.to_csv('prediction.csv',index=False)